In [2]:
from kfp.components import load_component_from_url

Components

In [6]:
# TODO: These should be pulled from a more long-term home of 
# reusable components
copy_to_minio_op = load_component_from_url('https://raw.githubusercontent.com/StatCan/kubeflow-mlops/scribner-iowa-staging/components/copy_to_minio.yaml')
copy_from_minio_op = load_component_from_url('https://raw.githubusercontent.com/StatCan/kubeflow-mlops/scribner-iowa-staging/components/copy_from_minio.yaml')

In [ ]:
def build_pipeline(container_tag: str = ":latest", **kwargs):

    # TODO: Add default minio creds?
    
    def training_pipeline(
        data_train: str,
        data_test: str,
        svm_C: float,
        svm_gamma: float,
        output_model: str,
        output_results: str,
    )
        """
        Pipeline for training the Iowa liquor categorization pipeline
        """

        operations = {}

        # FINISH THIS.  How do I get outputs from this?  Can't remember...  I think its just from .outputs and it knows?
        operations['get training data'] = copy_from_minio_op(
            minio_url=minio_url,
            minio_access_key=minio_access_key,
            minio_secret_key=minio_secret_key,
            minio_source=,
            local_destination=WHATEVER_I_WANT_TO_CALL_IT_LOCALLY,
        )